[Transformer 모델 설계]
우리는 "Encoder-Only Transformer" (BERT 스타일) 구조를 사용할 것입니다.
입력 (Input Sequence):
모양: (Batch, Sequence_Length, Feature_Dim)
Sequence_Length: 과거 10개의 이벤트 (LSTM보다 더 길게 봐도 됨)
Feature_Dim: 스텝마다 [공 위치, 골대 정보, 액션ID, + 주변 선수들 정보]
모델 구조:
Embedding Layer: 좌표와 액션ID 등을 고차원 벡터로 변환
Positional Encoding: 순서 정보 주입
Transformer Encoder: Self-Attention을 통해 "아, 3스텝 전의 패스가 중요했구나"를 파악
Output Head: 마지막 토큰의 정보를 받아서 (next_x, next_y) 예측
[작업 순서]
전처리: pre_test의 시퀀스 생성 로직을 가져오되, 주변 선수 정보도 포함시킵니다.
모델링: PyTorch로 간단한 Transformer 클래스를 만듭니다.
학습: GPU 서버(또는 로컬)에서 학습합니다. Transformer는 병렬 처리가 잘 돼서 LSTM보다 학습도 빠릅니다.
바로 Transformer 모델 코드를 짜드릴까요?

이 코드는 다음 단계로 구성됩니다:
데이터 전처리: 전체 데이터를 로드하고, 각 에피소드별로 시퀀스 데이터(Sequence Data)를 만듭니다. 피처에는 [x, y, 골대거리, 골대각도, 액션ID]를 포함합니다. (주변 선수 정보는 일단 데이터가 무거워질 수 있으니, 1차적으로 가벼운 버전으로 먼저 성능을 봅니다.)
Transformer 모델 정의: PyTorch의 nn.TransformerEncoder를 사용하여 시계열 패턴을 학습하는 모델을 만듭니다.
학습 및 추론: 모델을 학습시키고, test.csv에 대한 제출 파일을 생성합니다.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import math

# ----------------------------------------------------------
# 1. 데이터 전처리 (시퀀스 생성)
# ----------------------------------------------------------
print("데이터 로드 중...")
train_df = pd.read_csv("../open_track1/train.csv")

# 정규화 (Normalization)
train_df['start_x'] /= 105.0
train_df['start_y'] /= 68.0
train_df['end_x'] /= 105.0
train_df['end_y'] /= 68.0
# Action ID 정규화 (최대값 약 60)
train_df['action_id'] = train_df['action_id'].fillna(0) / 60.0 

def create_transformer_dataset(df, seq_len=10): # 시퀀스 길이 10으로 증가
    sequences = []
    targets = []
    GOAL_X, GOAL_Y = 1.0, 0.5
    
    grouped = df.groupby('game_episode')
    for name, group in tqdm(grouped, desc="시퀀스 생성"):
        group = group.sort_values('time_seconds')
        
        coords = group[['start_x', 'start_y']].values
        actions = group['action_id'].values.reshape(-1, 1)
        
        # 골대 정보
        dist = np.sqrt((coords[:,0]-GOAL_X)**2 + (coords[:,1]-GOAL_Y)**2).reshape(-1,1)
        angle = np.arctan2(coords[:,1]-GOAL_Y, coords[:,0]-GOAL_X).reshape(-1,1)
        
        # Feature: [x, y, dist, angle, action_id] (5 dim)
        features = np.hstack([coords, dist, angle, actions])
        
        # Target
        last_action = group.iloc[-1]
        target = [last_action['end_x'], last_action['end_y']]
        
        # Padding
        if len(features) >= seq_len:
            seq = features[-seq_len:]
        else:
            padding = np.zeros((seq_len - len(features), 5))
            seq = np.vstack([padding, features])
            
        sequences.append(seq)
        targets.append(target)
        
    return np.array(sequences), np.array(targets)

# 데이터셋 생성
X_seq, y_seq = create_transformer_dataset(train_df, seq_len=10)
print(f"데이터 Shape: {X_seq.shape}") # (N, 10, 5)

# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Loader
train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train)), batch_size=64, shuffle=True)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)


# ----------------------------------------------------------
# 2. Transformer 모델 정의 (Time-Series Transformer)
# ----------------------------------------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class SoccerTransformer(nn.Module):
    def __init__(self, feature_size=5, d_model=128, nhead=4, num_layers=2, dropout=0.1):
        super(SoccerTransformer, self).__init__()
        self.d_model = d_model
        
        # 입력 임베딩 (5 -> 128)
        self.embedding = nn.Linear(feature_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        # Transformer Encoder Layer
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=256, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        # 출력층
        self.fc_out = nn.Linear(d_model, 2) # (x, y) 예측
        
    def forward(self, src):
        # src shape: (Batch, Seq_Len, Feature)
        # Transformer requires: (Seq_Len, Batch, Feature)
        src = src.permute(1, 0, 2) 
        
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        
        output = self.transformer_encoder(src)
        
        # 마지막 시점의 출력만 사용
        # output shape: (Seq_Len, Batch, d_model)
        last_output = output[-1, :, :]
        
        pred = self.fc_out(last_output)
        return pred

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

model = SoccerTransformer().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print(model)

# ----------------------------------------------------------
# 3. 학습
# ----------------------------------------------------------
print("\n[Transformer 학습 시작]")
epochs = 50

X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    # 검증
    model.eval()
    with torch.no_grad():
        val_out = model(X_val_tensor).cpu().numpy()
        y_val_np = y_val_tensor.cpu().numpy()
        
        diff_x = (val_out[:,0] - y_val_np[:,0]) * 105.0
        diff_y = (val_out[:,1] - y_val_np[:,1]) * 68.0
        val_score = np.mean(np.sqrt(diff_x**2 + diff_y**2))
    
    scheduler.step(val_score)
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1:02d}, Loss: {running_loss/len(train_loader):.5f}, Val Score: {val_score:.4f}")

# ----------------------------------------------------------
# 4. 추론 및 제출
# ----------------------------------------------------------
print("\n[추론 시작]")
test_meta = pd.read_csv("../open_track1/test.csv")
base_path = "../open_track1"
GOAL_X, GOAL_Y = 1.0, 0.5

final_predictions_x = []
final_predictions_y = []

model.eval()
with torch.no_grad():
    for idx, row in tqdm(test_meta.iterrows(), total=len(test_meta)):
        file_path = os.path.join(base_path, row['path'][2:])
        if os.path.exists(file_path):
            temp_df = pd.read_csv(file_path)
            
            # 전처리 동일하게 적용
            temp_df['start_x'] /= 105.0
            temp_df['start_y'] /= 68.0
            if 'action_id' in temp_df.columns:
                temp_df['action_id'] = temp_df['action_id'].fillna(0) / 60.0
            else:
                temp_df['action_id'] = 0
                
            coords = temp_df[['start_x', 'start_y']].values
            actions = temp_df['action_id'].values.reshape(-1, 1)
            
            dist = np.sqrt((coords[:,0]-GOAL_X)**2 + (coords[:,1]-GOAL_Y)**2).reshape(-1,1)
            angle = np.arctan2(coords[:,1]-GOAL_Y, coords[:,0]-GOAL_X).reshape(-1,1)
            
            features = np.hstack([coords, dist, angle, actions])
            
            if len(features) >= 10:
                seq = features[-10:]
            else:
                padding = np.zeros((10 - len(features), 5))
                seq = np.vstack([padding, features])
            
            input_tensor = torch.FloatTensor(seq).unsqueeze(0).to(device)
            pred = model(input_tensor).cpu().numpy()
            
            final_predictions_x.append(pred[0,0] * 105.0)
            final_predictions_y.append(pred[0,1] * 68.0)
        else:
            final_predictions_x.append(52.5)
            final_predictions_y.append(34.0)

sub = pd.read_csv("../open_track1/sample_submission.csv")
sub['end_x'] = final_predictions_x
sub['end_y'] = final_predictions_y
filename = "submission_transformer_v1.csv"
sub.to_csv(filename, index=False)
print(f"저장 완료: {filename}")

데이터 로드 중...


시퀀스 생성: 100%|██████████| 15435/15435 [00:08<00:00, 1924.49it/s]
c:\Users\user\.conda\envs\soccer\lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


데이터 Shape: (15435, 10, 5)
Using Device: cpu
SoccerTransformer(
  (embedding): Linear(in_features=5, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=128, out_features=2, bias=True)
)

[Transformer 학습 시작]
